In [1]:
import numpy as np # Successfully installed numpy-1.23.5
import pandas as pd # Successfully installed pandas-1.3.5
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Dataset
from torch_geometric.data import Data
from torch_geometric.nn import global_mean_pool
# the temporal package is not available TODO: find what the issue is
# from torch_geometric_temporal.nn.recurrent import A3TGCN2 # torch-geometric-temporal 0.54.0 requires pandas<=1.3.5, but you have pandas 2.2.3 which is incompatible.
#from torch_geometric_temporal.signal import temporal_signal_split
# import 
# Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# from stellargraph import StellarGraph
# import custom functions from utils.py
# from utils import *
# os.getcwd() # double check current working directory

In [ ]:
# C:\Users\User1\Documents\GitHub\CAN-Graph\notebooks

In [2]:
# .. goes up one level in directory
path = r'../datasets/Car-Hacking Dataset/Fuzzy_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/DoS_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/gear_dataset.csv'
path = r'../datasets/Car-Hacking Dataset/RPM_dataset.csv'
df = pd.read_csv(path)
df.columns = ['Timestamp', 'CAN ID','DLC','Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8', 'label'] 

In [3]:
'''
Temporal graph dataset: a graph that changes over time
Graph type: Homogeneous graph
Nodes: CAN ID
Edges: The next CAN ID in the sequence
Node Features (optional): The data in the CAN ID

Steps:
1. identify node properties
2. Edges: How to connect nodes?
3. Extract labels

Convert to pytorch geometric format
One graph to another graph: 200 messages to 1 graph

There is also pytorch geometric temporal package

# build a baseline model RF, SVM, etc

# fraud datasets may have incorrect labels -> use label smoothing
Featureless nodes: use random numbers

make a GCN that uses labels only, and one that takes an average of data form its neighbors as well
Graph Attention Networks (GAT)

Idea: could I use FL on the different car datasets, share those weights and perhaps it could generalize better. A practical use case would be between different companies, where they don't want to share their data
but it is in their best interest to have the best performing cybersecurity model.

https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.explain.algorithm.GNNExplainer.html
'''

"\nTemporal graph dataset: a graph that changes over time\nGraph type: Homogeneous graph\nNodes: CAN ID\nEdges: The next CAN ID in the sequence\nNode Features (optional): The data in the CAN ID\n\nSteps:\n1. identify node properties\n2. Edges: How to connect nodes?\n3. Extract labels\n\nConvert to pytorch geometric format\nOne graph to another graph: 200 messages to 1 graph\n\nThere is also pytorch geometric temporal package\n\n# build a baseline model RF, SVM, etc\n\n# fraud datasets may have incorrect labels -> use label smoothing\nFeatureless nodes: use random numbers\n\nmake a GCN that uses labels only, and one that takes an average of data form its neighbors as well\nGraph Attention Networks (GAT)\n\nIdea: could I use FL on the different car datasets, share those weights and perhaps it could generalize better. A practical use case would be between different companies, where they don't want to share their data\nbut it is in their best interest to have the best performing cybersecur

In [3]:
df['Node'] = df['CAN ID']
df['Edge'] = df['CAN ID'].shift(-1)

In [4]:
def hex_to_decimal(x):
    if x is None or x == 'None':
        return None
    try:
        return int(x, 16)
    except (ValueError, TypeError):
        return x


In [5]:
# hex_to_decimal = lambda y: int(y, 16)
# Need to encode the strings to integers
df = df.apply(lambda x: x.apply(hex_to_decimal))

In [6]:
# Drop the last row
df = df.drop(df.index[-1])

In [7]:

class TimeSeriesGraphDataset(Dataset):
    def __init__(self, time_series_data, window_size, stride):
        self.data = time_series_data
        self.window_size = window_size
        self.stride = stride
        self.graphs = self._create_graphs()

    def _create_graphs(self):
        graphs = []
        for i in range(0, len(self.data) - self.window_size + 1, self.stride):
            window = self.data[i:i+self.window_size]
            graph = self.create_single_graph(window)
            graphs.append(graph)
        return graphs

    def create_single_graph(self, window_data):
        x = torch.tensor(window_data, dtype=torch.float)
        # call the edge index function here
        edge_index = self._get_edge_index(window_data)
        # last column are the labels
        y = torch.tensor(window_data[:, -1], dtype=torch.long)
        return Data(x=x, edge_index=edge_index, y=y)
    
    def _get_edge_index(self, window_data: np.ndarray) -> torch.Tensor:
        # Assuming your data is in a list or numpy array called 'data'
        nodes = torch.tensor([row[0] for row in window_data], dtype=torch.long)
        edges = torch.tensor([row[1] for row in window_data], dtype=torch.long)

        edge_index = torch.stack([nodes, edges], dim=0)
        # I need to look into the utility of contiguous
        # edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        return edge_index

    def len(self):
        return len(self.graphs)

    def get(self, idx):
        return self.graphs[idx]

In [8]:
# import cProfile
# import pstats
# import io

# # Profile the TimeSeriesGraphDataset creation
# pr = cProfile.Profile()
# pr.enable()

# # Create the TimeSeriesGraphDataset
# dataset = TimeSeriesGraphDataset(arr, window_size=50, stride=1)

# pr.disable()
# s = io.StringIO()
# sortby = 'cumulative'
# ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
# ps.print_stats()
# print(s.getvalue())

In [9]:
df

,Timestamp,CAN ID,DLC,Data1,Data2,Data3,Data4,Data5,Data6,Data7,Data8,label,Node,Edge
0,1.478191e+09,399,8,254,59,0,0.0,0.0,60.0,0.0,0.0,R,399,608.0
1,1.478191e+09,608,8,25,34,34,48.0,255.0,143.0,110.0,63.0,R,608,672.0
2,1.478191e+09,672,8,96,0,131,29.0,150.0,2.0,189.0,0.0,R,672,809.0
3,1.478191e+09,809,8,220,184,126,20.0,17.0,32.0,0.0,20.0,R,809,1349.0
4,1.478191e+09,1349,8,216,0,0,131.0,0.0,0.0,0.0,0.0,R,1349,704.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621695,1.478201e+09,790,8,5,33,48,10.0,33.0,30.0,0.0,111.0,R,790,399.0
4621696,1.478201e+09,399,8,254,89,0,0.0,0.0,65.0,0.0,0.0,R,399,608.0
4621697,1.478201e+09,608,8,24,33,33,48.0,8.0,143.0,109.0,25.0,R,608,672.0
4621698,1.478201e+09,672,8,36,0,154,29.0,151.0,2.0,189.0,0.0,R,672,809.0


In [10]:
# Drop rows where 'Data3' contains 'R'
df_dropped = df[df['Data3'] != 'R']
df= df_dropped

In [ ]:
# # Identify rows where 'Data3' contains 'R'
# mask = df['Data3'] == 'R'

# # Pad these rows with zeros and add 'R' label at the end
# df.loc[mask, :] = df.loc[mask, :].fillna(0)
# df.loc[mask, 'label'] = 'R'

C:\Users\User1\AppData\Local\Temp\ipykernel_16060\40276460.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[mask, :] = df.loc[mask, :].fillna(0)


In [11]:
df['label'] = df['label'].replace({'R': 0, 'T': 1})

C:\Users\User1\AppData\Local\Temp\ipykernel_20304\4091684753.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'R': 0, 'T': 1})
C:\Users\User1\AppData\Local\Temp\ipykernel_20304\4091684753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].replace({'R': 0, 'T': 1})


In [12]:
# Replace NaN values with zero
df = df.fillna(0)

C:\Users\User1\AppData\Local\Temp\ipykernel_20304\63676179.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


In [13]:
# Function to pad rows with zeros and add 'R' label at the end
def pad_row(row):
    if row['Data3'] == 'R':
        return pd.Series(np.append(row.fillna(0).values, 'R'))
    return row

# Apply the function to each row
# df_padded = df.apply(pad_row, axis=1)

# # Set pandas display option to show all rows
# # pd.set_option('display.max_rows', None)

# # Print the DataFrame
# print(df_padded)

In [14]:
# # Extract edge indices
edge_index = torch.tensor(df[['Node', 'Edge']].values, dtype=torch.long)

# # Extract node features (assuming features are in columns 'feature1', 'feature2', ..., 'featureN')
node_features = torch.tensor(df[['Data1','Data2','Data3','Data4','Data5','Data6','Data7','Data8',]].values, dtype=torch.float)

y = torch.tensor(df['label'], dtype=torch.long)
# # Create a Data object
data = Data(x=node_features, edge_index=edge_index, y=y)

In [15]:
data

Data(x=[4580224, 8], edge_index=[4580224, 2], y=[4580224])

In [16]:
'''' From GitHub Copilot
Key Attributes of the Data Class:
x: Node feature matrix with shape [num_nodes, num_node_features]. Each row corresponds to the feature vector of a node.
edge_index: Graph connectivity in COO format with shape [2, num_edges]. Each column represents an edge between two nodes.
edge_attr (optional): Edge feature matrix with shape [num_edges, num_edge_features]. Each row corresponds to the feature vector of an edge.
y (optional): Graph-level or node-level labels.
pos (optional): Node position matrix with shape [num_nodes, num_dimensions]. Useful for visualization or spatial graphs.
batch (optional): Batch vector, which assigns each node to a specific graph in a mini-batch.
'''
# label logic - if T in window then label is 1, else 0. Start simple and build out from there
# x = [# nodes, # features], for my case [200, 10]
# edge_index = [2, # edges], for my case [2, 199]
# y = [# nodes, # labels], for my case [200, 1]
arr = df[['CAN ID', 'Node', 'Edge']].to_numpy()
# maybe a stride of 25 to speed up the process?? At least to start
# stride of 1 was over 10 GBs, so will increase to 25
dataset = TimeSeriesGraphDataset(arr, window_size=50, stride=25)

In [17]:
path = r'../datasets/dataset_rpms_25.pt'
torch.save(dataset, path)

In [22]:
dataset

AttributeError: 'TimeSeriesGraphDataset' object has no attribute '_indices'

In [21]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
class AnomalyGNN(nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(AnomalyGNN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.classifier = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Aggregate node features
        return self.classifier(x).squeeze()


num_epochs = 100
dataset = dataset
# Define the split ratio
train_ratio = 0.8
test_ratio = 0.2

# Calculate the number of samples for training and testing
# num_samples = len(dataset)
# num_train = int(train_ratio * num_samples)
# num_test = num_samples - num_train

# Split the dataset
# train_dataset, test_dataset = random_split(dataset, [num_train, num_test])
loader = DataLoader(dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

model = AnomalyGNN(num_node_features=50, hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(num_epochs):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()





AttributeError: 'TimeSeriesGraphDataset' object has no attribute '_indices'

In [ ]:
model.eval()
with torch.no_grad():
    for batch in test_loader:
        predictions = torch.sigmoid(model(batch.x, batch.edge_index))
        anomalies = predictions > 0.5  # Binary classification threshold